## Implementation of Likelihood Weighting to approximate probability queries from a Bayesian Network.

In [ ]:
import random
import copy
import numpy as np
from bayesian_network_helper import topologically_sort
from calculations_helper import find_corresponding_rows

def estimate_likelihood_weighting(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Use likelihood weighting to estimate the probability distribution of the given query variables

    Args:
        iterations (int): number of iterations where the variables are sampled and their counts are updated accordingly
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables and their values

    Returns:
        np.array: resulting probability distribution estimate for the query variables
    """
    prob_distribution = np.zeros(shape=1<<len(queries))
    
    # we're going to need to sort all of our variables topologically, because that's the order that we'll need to set their values to
    sorted_vars = topologically_sort(network=network)
    
    for _ in range(iterations):
        current_evidence = copy.deepcopy(evidence)
        # go through each non-evidence variable and assign it a value randomly according to its probability
        weight = 1.0
        for var in sorted_vars:
            probabilities = [pair[1] for pair in network[str(var)]["prob"]]
            parents = network[str(var)]["parents"]
            prob_true = 0.0
            if len(parents) > 0:
                # look at the parents to determine probability of being true
                row = find_corresponding_rows([1 if current_evidence[p] else 0 for p in parents], parents, parents)[0]
                prob_true += probabilities[row]
            else:
                prob_true += probabilities[0]

            # if the variable is in evidence, update the weight
            if var in evidence.keys():
                weight *= prob_true if evidence[var] else (1 - prob_true)
            else: # otherwise set in the current evidence the value of this variable to be according to the probability given ancestors' values
                current_evidence[var] = random.random() < prob_true
            
        query_values = [1 if current_evidence[q] else 0 for q in queries]
        # find the entry in our probability distribution that corresponds with this combination of query values
        posn = find_corresponding_rows(query_values, queries, queries)[0]
        # update the distribution accordingly
        prob_distribution[posn] += 1.0 * weight

    return prob_distribution / np.sum(prob_distribution) # normalization into a probability

In [70]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

[0.69161109 0.30838891]


In [71]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

[0.33663523 0.09813912 0.11899487 0.44623077]


In [72]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

[0.87712878 0.12287122]
